<a href="https://colab.research.google.com/github/LukasPurbaW/10_Days_of_Deep_Learning/blob/main/Day_10_Stock_Price_Prediction_with_LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Predict stock prices  with Long short-term memory (LSTM)

In [ ]:
!pip -qq install pandas-datareader

In [ ]:
!apt -qq install graphviz

In [ ]:
!pip -qq install pydot pydot-ng
!pip -qq install --upgrade pandas
!pip -qq install --upgrade pandas-datareader

## Loading the dataset
I use pandas-datareader to get the historical stock prices from Yahoo! finance. For this example, I get only the historical data till the end of *training_end_data*.  

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from pandas_datareader import data as pdr
import fix_yahoo_finance as yf
import datetime

In [ ]:
tickers = 'AAPL'

start_date = '1980-12-01'
end_date = '2018-12-31'

stock_data = pdr.get_data_yahoo(tickers, start_date, end_date)
stock_data.head(5)

In [ ]:
stock_data.describe()

In [ ]:
stock_data_len = stock_data['Close'].count()
print(stock_data_len)
# Close price means the price at the end of the day

In [ ]:
close_prices = stock_data.iloc[:, 1:2].values
print(close_prices)

In [ ]:
stock_data.isna().sum()

Some of the price are no avaliable during holiday, so we have to clean it first. For this reason, we can fill the missing prices with the latest available prices

In [ ]:
all_bussinessdays = pd.date_range(start=start_date, end=end_date, freq='B')
print(all_bussinessdays)

In [ ]:
close_prices = stock_data.reindex(all_bussinessdays)
close_prices = stock_data.fillna(method='ffill')

In [ ]:
stock_data.isna().sum()

In [ ]:
close_prices.head(10)

The dataset is now complete and free of missing values. Let's have a look to the data frame summary:

## Feature scaling

In [ ]:
training_set = close_prices.iloc[:, 1:2].values

In [ ]:
print(training_set)

In [ ]:
from sklearn.preprocessing import MinMaxScaler
sc = MinMaxScaler(feature_range = (0, 1))
training_set_scaled = sc.fit_transform(training_set)
print(training_set_scaled.shape)

LSTMs expect the data in a specific format, usually a 3D tensor. I start by creating data with 60 days and converting it into an array using NumPy. Next, I convert the data into a 3D dimension array with feature_set samples, 60 days and one feature at each step.

In [ ]:
features = []
labels = []
for i in range(60, stock_data_len):
    features.append(training_set_scaled[i-60:i, 0])
    labels.append(training_set_scaled[i, 0])

features = np.array(features)
labels = np.array(labels)

features = np.reshape(features, (features.shape[0], features.shape[1], 1))

In [ ]:
print(labels)

In [ ]:
print(features)

Feature tensor with three dimension: features[0] contains the ..., features[1] contains the last 60 days of values and features [2] contains the  ...

In [ ]:
print(features.shape)

## Create the LSTM network
Let's create a sequenced LSTM network with 50 units. Also the net includes some dropout layers with 0.2 which means that 20% of the neurons will be dropped.

In [ ]:
import tensorflow as tf

In [ ]:
model = tf.keras.models.Sequential([
    tf.keras.layers.LSTM(units = 50, return_sequences = True, input_shape = (features.shape[1], 1)),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.LSTM(units = 50, return_sequences = True),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.LSTM(units = 50, return_sequences = True),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.LSTM(units = 50),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(units = 1)
])

In [ ]:
print(model.summary())

## Load the Colab TensorBoard extention and start TensorBoard inline

In [ ]:
#%load_ext tensorboard.notebook
#%tensorboard --logdir {LOG_BASE_DIR}

The model will be compiled and optimize by the adam optimizer and set the loss function as mean_squarred_error

In [ ]:
model.compile(optimizer = 'adam', loss = 'mean_squared_error')

In [ ]:
from time import time
start = time()
history = model.fit(features, labels, epochs = 20, batch_size = 32, verbose = 1)
end = time()

In [ ]:
print('Total training time {} seconds'.format(end - start))

In [ ]:
#  [samples, days, features]
print(features.shape)

In [ ]:
testing_start_date = '2019-01-01'
testing_end_date = '2019-04-10'

test_stock_data = pdr.get_data_yahoo(tickers, testing_start_date, testing_end_date)

In [ ]:
test_stock_data.tail()

In [ ]:
test_stock_data_processed = test_stock_data.iloc[:, 1:2].values
print(test_stock_data_processed.shape)

In [ ]:
all_stock_data = pd.concat((stock_data['Close'], test_stock_data['Close']), axis = 0)
inputs = all_stock_data[len(all_stock_data) - len(test_stock_data) - 60:].values
inputs = inputs.reshape(-1,1)
inputs = sc.transform(inputs)

In [ ]:
X_test = []
for i in range(60, 129):
    X_test.append(inputs[i-60:i, 0])

In [ ]:
X_test = np.array(X_test)
X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))
predicted_stock_price = model.predict(X_test)
predicted_stock_price = sc.inverse_transform(predicted_stock_price)

In [ ]:
plt.figure(figsize=(10,6))  
plt.plot(test_stock_data_processed, color='blue', label='Actual Apple Stock Price')  
plt.plot(predicted_stock_price , color='red', label='Predicted Apple Stock Price')  
plt.title('Apple Stock Price Prediction')  
plt.xlabel('Date')  
plt.ylabel('Apple Stock Price')  
plt.legend()  
plt.show()  

## Download the model and the weights

In [ ]:
from google.colab import files

In [ ]:
model_json = model.to_json()
with open("model.json", "w") as json_file:
  json_file.write(model_json)

In [ ]:
files.download("model.json")

In [ ]:
model.save('weights.h5')
files.download('weights.h5')